## This part is dedicated to implement a blast-wave model and deduce paramters dependency from centrality, pp and phi meson production.

In [20]:
# I will import data that I already used, and then fit them with a levy functi
# on. Finally i will extand the range of the fit, calculate the integral on this part and the propagation of error on the integral. Finally I will reproduce the method for every centrality

In [21]:
import ROOT as r
import numpy as np
import math 
import matplotlib.pyplot as plt

In [22]:
def reposition_data(fitfunction, data):
    
    a=0
    list_newx = np.zeros(data.GetNbinsX()+1)
    
    for i in range(1, data.GetNbinsX()+1):

        width = data.GetBinWidth(i)
        
        a = data.GetBinLowEdge(i)
        b = a + width
    
        integral = fitfunction.Integral(a,b)
        mean = integral/width
        
        new_x = fitfunction.GetX(mean, a, b)

        list_newx[i] = new_x

    return list_newx

In [23]:
##Move the point of the  bins on the good localization in a new Tgraph, from a plot and a fitting function

#DEF FUNCTION

def x_reposition(data, fit_function):


    new_xlist = reposition_data(fit_function, data)            
    graph = r.TGraphAsymmErrors(data.GetNbinsX())

    for i in range (1,data.GetNbinsX()+1):
    
        y  = data.GetBinContent(i)
        x  = new_xlist[i]
        ey = data.GetBinError(i)/2
        xm = data.GetBinLowEdge(i) + data.GetBinWidth(i)/2
        deltax = xm - x
    
        if deltax > 0:
            exl = data.GetBinWidth(i)/2 - deltax
            exr = data.GetBinWidth(i)/2 + deltax
        else:
            exl = data.GetBinWidth(i)/2 + deltax
            exr = data.GetBinWidth(i)/2 - deltax
        
        graph.SetPoint(i, x, y)
        graph.SetPointError(i, exl, exr, ey, ey)

    
    return graph

In [24]:

PbPb_pp_file = r.TFile("ROOTfile/PbPb_pp5tev.root")
Histo = PbPb_pp_file.Get("Table 5")

Y1 = Histo.Get("Hist1D_y1;1")
e1 = Histo.Get("Hist1D_y1_e1")
e2 = Histo.Get("Hist1D_y1_e2")
e3 = Histo.Get("Hist1D_y1_e3")

# This part is to avoid the kernel to crash, so I fix the memory position of my base plot
y1_plot  = Y1.Clone("y1_plot")
y1_plot.SetDirectory(0)


In [25]:
#Here I calculated the squared sum of uncorrelated uncertainties and place it for each bin

for i in range(1, y1_plot.GetNbinsX()+1): #nb of bins
    err = math.sqrt( e1.GetBinContent(i)**2)# +  e3.GetBinContent(i)**2 ) ##We didn't take now into account correlated uncertainties
    y1_plot.SetBinError(i, err) 


In [26]:
##I want to estimate dN/dY for 0-5% centrality

dndy_estimate = 0

for i in range(1, y1_plot.GetNbinsX()+1):
    dndy_estimate = dndy_estimate + y1_plot.GetBinContent(i)*y1_plot.GetBinWidth(i)

print(dndy_estimate)


72.39154661211215


In [27]:
#new def of the function
#mp = 0.938272
#
#def blastwave_tf1(x_abs, par):
#    
#    pt = x_abs[0]
#    
#    A = par[0]
#    Tkin = par[1]
#    beta_s = par[2]
#    n = par[3]
#    m = par[4]
#
#
 #   mT = math.sqrt(m*m + pt*pt)
#
 #   if Tkin <= 0 or beta_s <= 0 or beta_s >= 1 or n < 0 or m < 0:
  #      return 0.0
#
 #       
  #  def function(x_loc, par_loc):
#
 #       x = x_loc[0]
  #      
   #     beta = beta_s * (x ** n)
    #    if beta >= 1.0:
     #       beta = 0.999999999 
#
 #       rho = math.atanh(beta)  
#
 #       argI0 = (pt * math.sinh(rho)) / Tkin
  #      argK1 = (mT * math.cosh(rho)) / Tkin
#
 #       I0 = r.TMath.BesselI0(argI0)
  #      K1 = r.TMath.BesselK1(argK1)
#
 #       return x * I0 * K1 
  #  
#
 #   function_tf1 = r.TF1("fct_tf1", 0.0, 1.0, 0)
  #  function_tf1.SetNpx(500)
   # integral = function_tf1.Integral(0.0, 1.0)
#
 #   return A * mT * pt * integral
  #  

In [28]:
#fit_min, fit_max = 0.3 , 3
#
#bw_tf1 = r.TF1("bw", blastwave_tf1, fit_min, fit_max, 5)
#bw_tf1.SetParNames("A", "Tkin", "beta_s", "n", "m")
#
#bw_tf1.SetParameter(0, 1.0) #A
#bw_tf1.SetParameter(1, 0.2) #Tkin
#bw_tf1.SetParameter(2, 0.5)
#bw_tf1.SetParameter(3, 1)
#bw_tf1.SetParameter(4, 93827)#
#
#bw_tf1.FixParameter(3, 1)#
#
#
#bw_tf1.SetParLimits(0, 0.0, 1e12)    
#bw_tf1.SetParLimits(1, 0.02, 0.30)    
#bw_tf1.SetParLimits(2, 0.05, 0.95)    
#bw_tf1.SetParLimits(3, 0.0, 5.0)    
 
#bw_tf1.FixParameter(4, 0.93827)      

In [29]:
# vglobal variable used
_current_pT = 0.0
_current_mT = 0.0
_current_Tkin = 0.1
_current_beta_s = 0.6
_current_n = 1.0

_last_Tkin = 0.1
_last_beta = 0.6
_last_n = 1.0
_last_m = None
_last_norm = None

def function_tf1(x_abs, par):

    x = x_abs[0]

    beta = _current_beta_s * (x ** _current_n)
    if beta >= 1.0:
        beta = 0.999999999

    rho = math.atanh(beta)

    argI0 = (_current_pT * math.sinh(rho)) / _current_Tkin
    argK1 = (_current_mT * math.cosh(rho)) / _current_Tkin

    I0 = r.TMath.BesselI0(argI0)
    K1 = r.TMath.BesselK1(argK1)

    return x * I0 * K1

In [30]:
function_tf1 = r.TF1("function_tf1", function_tf1, 0.0, 1.0, 0)

from array import array

# nombre de points de quadrature
nb_sampling_point = 64


# tableau des abscisses xi et poids wi
xg = array('d')      
wg = array('d')
xg_pt = array('d')
wg_pt = array('d')

for i in range(nb_sampling_point):
    xg.append(0.0)
    wg.append(0.0)
    xg_pt.append(0.0)
    wg_pt.append(0.0)

function_tf1.CalcGaussLegendreSamplingPoints(nb_sampling_point, xg, wg, 1e-12)
function_tf1.CalcGaussLegendreSamplingPoints(nb_sampling_point, xg_pt, wg_pt, 1e-12)

In [ ]:
def blastwave_tf1(x_abs, par):

    global _current_pT, _current_mT, _current_Tkin, _current_beta_t, _current_n
    global _last_Tkin, _last_beta, _last_n, _last_m, _last_norm

    pt = x_abs[0]
    
    dNdy = par[0]
    Tkin = par[1]
    beta_t = par[2]
    n = par[3]
    m = par[4]
    

    if Tkin <= 0 or beta_t <= 0 or beta_t >= 1 or n < 0 or m < 0:
        return 0.0

    mT = math.sqrt(m*m + pt*pt)

    _current_pT = pt
    _current_mT = mT
    _current_Tkin = Tkin
    _current_beta_t = beta_t
    _current_n = n

    integral = function_tf1.IntegralFast(nb_sampling_point, xg, wg, 0.0, 1.0)
    
    f_pt = mT * pt * integral ## Calculation of the blast- wave function, but without normalization (i want to extract dNdy to keep physics, and before my normalization was diverging during the fit)


    if (_last_Tkin != Tkin) or (_last_beta != beta_t) or (_last_n != n) or (_last_m != m):

        s = 0.0

        for i in range(nb_sampling_point):

            # the gauss-legendre weight are 
            z = xg_pt[i]
            w = wg_pt[i]

            p = 0.5*(fit_max-fit_min)*z + 0.5*(fit_max+fit_min)

            mT_p = math.sqrt(m*m + p*p)

            _current_pT = p
            _current_mT = mT_p
            _current_Tkin = Tkin
            _current_beta_t = beta_t
            _current_n = n

            integral_x_p = function_tf1.IntegralFast(nb_sampling_point, xg, wg, 0.0, 1.0)
            f_p = mT_p * p * integral_x_p

            s = s + w * f_p

        _last_norm = 0.5*(fit_max-fit_min)*s

        _last_Tkin = Tkin
        _last_beta = beta_t
        _last_n = n
        _last_m = m

    if _last_norm <= 0:
        return 0.0
    
    return dNdy * f_pt / _last_norm
    
    




In [44]:
fit_min, fit_max = 0.3, 3

bw_tf1 = r.TF1("bw_tf1", blastwave_tf1, fit_min, fit_max, 5)

bw_tf1.SetParNames("dNdy","Tkin","beta_t","n","m")
bw_tf1.SetParameters(90.0, 0.2, 0.6, 0.7, 0.938)

#bw_tf1.SetParLimits(0,0.02,0.15)


#bw_tf1.SetParLimits(1,0.02,0.15)
bw_tf1.SetParLimits(2,0.05,0.95)

bw_tf1.SetParLimits(3, 0.6, 0.9)
#bw_tf1.SetParLimits(4,0.5,10)

bw_tf1.FixParameter(4,0.938)

In [45]:
fit_res = y1_plot.Fit(bw_tf1, "RS0")

****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      19388.2
NDf                       =           25
Edm                       =  6.15361e-06
NCalls                    =          135
dNdy                      =      65.8011   +/-   0.0506183   
Tkin                      =     0.393595   +/-   0.00044147  
beta_t                    =          0.6   +/-   0.708893     	 (limited)
n                         =          0.6   +/-   0.000125597  	 (limited)
m                         =        0.938                      	 (fixed)


In [85]:
Final_pp_data = x_reposition(y1_plot,bw_tf1)

In [86]:
c = r.TCanvas("c", "Data + Levy Fit with modified x")
c.SetGrid()


Final_pp_data.SetTitle("pt distribution of p+p- in PbPb collision at 5 Tev fitted by blast wave model")
Final_pp_data.GetXaxis().SetTitle("pt [Gev/c]")
Final_pp_data.GetYaxis().SetTitle("(1/Nev)*D²(N)/DptDyrap [(Gev/c)^⁻1]")

Final_pp_data.Draw("AP") 

bw_tf1.SetLineColor(r.kRed)
bw_tf1.Draw("SAME")

legend = r.TLegend(0.7,0.7,0.9,0.9)
legend.AddEntry(Final_pp_data, "pp data", "lep")
legend.AddEntry(bw_tf1, "Blast-wave fit")
legend.Draw()

CHI = str(bw_tf1.GetChisquare())[:8]
NDF = str(bw_tf1.GetNDF())

Text = r.TPaveText(6,10,12,15)
Text.AddText(f" #chi^{{2}} = {CHI}")
Text.AddText(f" NdF = {NDF} ")
Text.Draw()


c.Draw()
#c2.SaveAs("y1_fit_levyRE0_free-dNdy.pdf")
print(fit_res)


****************************************
         Invalid FitResult  (status = 4 )
****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      134.149
NDf                       =           25
Edm                       =     0.127406
NCalls                    =         1632
A                         =  2.40213e+07   +/-   2.61374e+06 
Tkin                      =     0.101153   +/-   0.0010582    	 (limited)
beta_s                    =     0.902502   +/-   0.000996635  	 (limited)
n                         =     0.750071   +/-   0.00116577   	 (limited)
m                         =        0.938                      	 (fixed)



Warning in <TCanvas::Constructor>: Deleting canvas with same name: c


In [87]:
## How to reduce A ?

In [71]:
bw_tf1.SetParameter(0, 300)
bw_tf1.SetParLimits(0,1,2000)

bw_tf1.SetParameter(4,0.938)
bw_tf1.SetParLimits(4,0.5,10)



In [72]:
fit_res = y1_plot.Fit(bw_tf1, "RS0")

****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      45307.5
NDf                       =           24
Edm                       =  1.75554e-07
NCalls                    =          172
A                         =         2000   +/-   0.142322     	 (limited)
Tkin                      =          0.2   +/-   3.2688e-06   	 (limited)
beta_s                    =         0.05   +/-   0.0614831    	 (limited)
n                         =            2   +/-   1.07002      	 (limited)
m                         =          0.5   +/-   2.36626e-05  	 (limited)
